In [5]:
from graphics import *
import numpy as np


In [6]:
def runRotasiSembarang(list_point):
    """
    PROGRAM UTAMA
    Fungsi untuk menampilkan rotasi balok
    
    Args: 
        list_point (list atau np.array) : List titik sudut dari bangun ruang
        
    Returns:
        None
    """
    
    # Setup window
    win = GraphWin('Rotasi terhadap sembarang sumbu', 600,600)
    win.setCoords(-6,-6,6,6)
    
    # Menulis judul
    title = Text(Point(0, 5.2), 'Memutar balok terhadap sembarang sumbu')
    title.draw(win)
    
    # Menggambar posisi awal balok
    corners, skeletons = draw(win, list_point, keep=True)
    
    # Masukkan titik pangkal sumbu putar
    teksInputPangkal = Text(Point(-2.5,-3.6),"Titik pangkal sumbu\n (format: x,y,z)") #
    entriPangkal = Entry(Point(-2.5,-4.4), 8)
    entriPangkal.setFill('white')
    teksInputPangkal.draw(win)
    entriPangkal.draw(win)

    # Masukkan titik ujung sumbu putar
    teksInputUjung = Text(Point(2.5,-3.6),"Titik ujung sumbu\n (format: x,y,z)")
    entriUjung = Entry(Point(2.5,-4.4),8)
    entriUjung.setFill('white')
    teksInputUjung.draw(win)
    entriUjung.draw(win)

    teksEnter = Text(Point(0, -5), "Bila sudah mengisi input, klik di mana saja pada window")
    teksEnter.draw(win)
    
    x1,y1,z1,x2,y2,z2 = None, None, None, None, None, None
    mintaInput = True
    while mintaInput:
        win.getMouse()
        x1,y1,z1 = entriPangkal.getText().split(',')
        x1 = float(x1)
        y1 = float(y1)
        z1 = float(z1)
        x2,y2,z2 = entriUjung.getText().split(',')
        x2 = float(x2)
        y2 = float(y2)
        z2 = float(z2)
        
        if (y1 != y2) and (z1 != z2):
            mintaInput = False
        else:
            teks=[]
            if y1==y2:
                txtWarning = ['y1', 'y2']
            else:
                txtWarning = ['z1', 'z2']
            warning = Text(Point(0,-2), f'Harap koordinat {txtWarning[0]} dan {txtWarning[1]} tidak sama')
            warning.draw(win)
            time.sleep(1.5)
            warning.undraw()
    
    teksInputPangkal.undraw()
    teksInputUjung.undraw()
    entriPangkal.undraw()
    entriUjung.undraw()

    if np.sqrt(x1**2+y1**2+z1**2) > np.sqrt(x2**2+y2**2+z2**2):
        x1,y1,z1,x2,y2,z2 = x2,y2,z2,x1,y1,z1
    
    
    # Menggambar sumbu putar
    gradienSumbu = (z2-z1)/(y2-y1)
    intercept = z1-gradienSumbu*y1
    
    sumbuExtend = Line(Point(-3*y2,-3*y2*gradienSumbu+intercept),Point(3*y2, 3*y2*gradienSumbu+intercept))
    sumbuExtend.setFill('pink')
    sumbuExtend.draw(win)
    
    sumbu = Line(Point(y1,z1),Point(y2,z2))
    sumbu.setFill('red')
    sumbu.draw(win)
    
    teksPangkal = Text(Point(y1, z1-0.5), f'{x1}, {y1}, {z1}')
    teksUjung = Text(Point(y2, z2-0.5), f'{x2}, {y2}, {z2}')
    teksPangkal.draw(win)
    teksUjung.draw(win)
    
#     teksPangkal = Text(Point(3.2,2.7), '3,3,3')
#     teksUjung = Text(Point(5.2,5.2), '5,5,5')
#     teksPangkal.draw(win)
#     teksUjung.draw(win)
    
    # Meminta input besar sudut yang akan dirotasi
    entriSudut = Entry(Point(0,-4), 5)
    teks = Text(Point(0, -3.6), 'Masukkan sudut putar (dalam derajat), lalu klik di manapun pada window')
    entriSudut.setFill('white')
    entriSudut.draw(win)
    teks.draw(win)
    
    # Menerima input sudut rotasi
    
    win.getMouse()  # Program perlu suatu trigger agar program mengambil input dari kolom entri, getMouse berperan sbg trigger
    sudut = int(entriSudut.getText())
    entriSudut.undraw()
    teks.undraw()
    teksEnter.undraw()
    
    # Menuliskan subtitle 'Sudut putar: '
    subtitle = Text(Point(0,-5.2), f'Sudut putar : {sudut}°')
    subtitle.draw(win)
    
    # Menghapus posisi awal balok
    for corner in corners: corner.undraw()
    for skeleton in skeletons: skeleton.undraw()
        
    # Merotasi balok
    for iterr in range(sudut):
        isFinished = True if (iterr==sudut-1) else False        
        draw(win, list_point, isFinished)
        list_point = rotateSembarang(list_point,x1,y1,z1,x2,y2,z2)
    
    # Program selesai
    win.getMouse()
    win.close()

In [7]:
def draw(window, list_point, keep=True):
    """
    Fungsi untuk Menggambar titik sudut dan garis kerangka balok
    
    Args: 
        windows (graphics.GraphWin): window dimana objek akan digambar
        list_point (list) : list titik sudut yang akan digambar
        keep (bool) : flag penentu apakah bangun yang telah digambar ingin disimpan atau tidak
        
    Returns:
        points (list of Point) : list dari Point yang telah digambar
        lines (list of Line) : list dari Line yang telah digambar
    """
    
    points = []  # list penampung semua objek Point
    lines = []  # list penampung semua objek Line
    
    # Menggambar titik sudut balok
    for i, koor_point in enumerate(list_point):
        point_temp = Point(koor_point[1],koor_point[2])  
        # Hanya mengambil koordinat y dan z dari koor_point
        # Hal ini karena layar desktop dispan oleh sumbu y dan z, sedangkan sumbu x ⟂ layar
        points.append(point_temp)
        points[i].draw(window)
    
    # Menggambar garis kerangka balok untuk sisi atas (A-B-C-D) dan bawah (E-F-G-H)
    bidangAtasLaluBawah = [0,4]
    for j in bidangAtasLaluBawah:
        for k in range(j, 4+j):
            line_temp = None
            if (k+1) != 4+j:
            # Garis (AB, BC, CD, dan EF, FG, GH)
                line_temp = Line(points[k], points[k+1])
                # print(f'point {k}:', k,',', k+1) --> troubleshooting, uncomment untuk melihat indeks line
            else:
            # Garis (DA dan HE)
                line_temp = Line(points[k], points[k-3])
                # print(f'point {k}:', k, ',', k-3) --> troubleshooting, uncomment untuk melihat indeks line
            lines.append(line_temp)
            
            lines[k].draw(window)
            
    # Menggambar garis kerangka balok untuk sisi samping: (A-E), (B-F), (C-G), (D-H)
    for l in range(4):
        # print(f'point {8+l}:', l,',',l+4) --> troubleshooting, uncomment untuk melihat indeks line
        line_temp = Line(points[l],points[l+4])
        lines.append(line_temp)
        lines[8+l].draw(window)
    
    # Jeda
    time.sleep(.03)
    
    # Untuk iterasi terakhir, gambar dibiarkan tetap di window
    # Jika bukan iter terakhir, gambar segera di-undraw
    if not keep:
        for point in points:
            point.undraw()
        for line in lines:
            line.undraw()
            
    return points, lines

In [8]:
def rotateSembarang(list_point, x1,y1,z1,x2,y2,z2):
    list_point_transpos = np.array(list_point).T
    
    T = np.array([[1,0,0,-x1],
                  [0,1,0,-y1],
                  [0,0,1,-z1],
                  [0,0,0,1]])
    
    dx = x2-x1
    dy = y2-y1
    dz = z2-z1
    
    beta = np.arctan(dx/dz)
    
    Ry_1 = np.array([[ np.cos(-beta), 0, np.sin(-beta), 0],
                     [             0, 1,             0, 0],
                     [-np.sin(-beta), 0, np.cos(-beta), 0],
                     [             0, 0,             0, 1]])
    
    temp = Ry_1 @ np.array([dx,dy,dz,1])
    miu = np.arctan(temp[1]/temp[2])
    
    Rx = np.array([[1,0,0,0],
                   [0,np.cos(miu), -np.sin(miu),0],
                   [0, np.sin(miu), np.cos(miu),0],
                   [0,0,0,1]])
    Rotasi = np.array([[np.cos(np.radians(1)), -np.sin(np.radians(1)),0,0],
                       [np.sin(np.radians(1)), np.cos(np.radians(1)), 0,0],
                       [0,                     0,                     1,0],
                       [0,                     0,                     0,1]])
    Rx_1 = np.linalg.inv(Rx)
    Ry = np.linalg.inv(Ry_1)
    T_1 = np.linalg.inv(T)
    
    M = T_1 @ Ry @ Rx_1 @ Rotasi @ Rx @ Ry_1 @ T
    return (M @ list_point_transpos).T

In [9]:
# Setup koordinat balok
A = np.array([ 1,-2, 1,1])
B = np.array([ 1, 2, 1,1])
C = np.array([-1, 2, 1,1])
D = np.array([-1,-2, 1,1])
E = np.array([ 1,-2,-1,1])
F = np.array([ 1, 2,-1,1])
G = np.array([-1, 2,-1,1])
H = np.array([-1,-2,-1,1])

# Matriks seluruh koordinat balok
points_awal = [A,B,C,D,E,F,G,H]

In [10]:
# Run program
runRotasiSembarang(points_awal)

In [11]:
import numpy as np

beta = np.radians(45)
Ry_1 = np.array([[ np.cos(-beta), 0, np.sin(-beta), 0],
                 [             0, 1,             0, 0],
                 [-np.sin(-beta), 0, np.cos(-beta), 0],
                 [             0, 0,             0, 1]])
p = np.array([2,2,2,1])
temp = Ry_1@p
np.arctan(temp[1]/temp[2])

0.6154797086703873

In [12]:
temp

array([0.        , 2.        , 2.82842712, 1.        ])

In [13]:
T = np.array([[1,0,0,-3],
              [0,1,0,-3],
              [0,0,1,-3],
              [0,0,0,1]])

In [14]:
Ry_1 @ T

array([[ 7.07106781e-01,  0.00000000e+00, -7.07106781e-01,
        -1.11022302e-16],
       [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
        -3.00000000e+00],
       [ 7.07106781e-01,  0.00000000e+00,  7.07106781e-01,
        -4.24264069e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])